## Create db and evaluate data quality

In [1]:
import pandas as pd

In [2]:
b = pd.read_json('brands.json',lines=True)
b['_id'] = b['_id'].apply(lambda x:x['$oid'])
b['cpg'] = b['cpg'].apply(lambda x:x['$id']['$oid'])
b.head()


,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [3]:
r = pd.read_json('receipts.json',lines=True)
r['_id'] = r['_id'].apply(lambda x:x['$oid'])
r['createDate'] = r['createDate'].apply(lambda x:pd.to_datetime(x['$date'],unit='ms') if x else x)
r['dateScanned'] = r['dateScanned'].apply(lambda x:pd.to_datetime(x['$date'],unit='ms') if x else x)
r['finishedDate'] = r['finishedDate'].apply(lambda x:pd.to_datetime(x['$date'],unit='ms') if type(x) != float else x)
r['modifyDate'] = r['modifyDate'].apply(lambda x:pd.to_datetime(x['$date'],unit='ms') if x else x)
r['pointsAwardedDate'] = r['pointsAwardedDate'].apply(lambda x:pd.to_datetime(x['$date'],unit='ms') if type(x) != float else x)
r['purchaseDate'] = r['purchaseDate'].apply(lambda x:pd.to_datetime(x['$date'],unit='ms') if type(x) != float else x)
# r = r.drop('rewardsReceiptItemList', axis=1)
r.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [4]:
r['rewardsReceiptStatus'].value_counts()

FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: rewardsReceiptStatus, dtype: int64

In [5]:
lst=[]
for k, v in r[~r['rewardsReceiptItemList'].isna()][['_id','rewardsReceiptItemList']].iterrows():
    i = v['rewardsReceiptItemList']
    for j in i:
        j['receipt_id'] = v['_id']
        lst.append(j)

In [6]:
item = pd.DataFrame(lst).set_index('receipt_id').reset_index()

In [7]:
item.head()

,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
item['barcode']

0               4011
1               4011
2       028400642255
3                NaN
4               4011
            ...     
6936      B07BRRLSVC
6937      B076FJ92M4
6938      B07BRRLSVC
6939      B076FJ92M4
6940      B07BRRLSVC
Name: barcode, Length: 6941, dtype: object

In [9]:
u = pd.read_json('users.json',lines=True)
u['_id'] = u['_id'].apply(lambda x:x['$oid'])
u['createdDate'] = u['createdDate'].apply(lambda x:pd.to_datetime(x['$date'],unit='ms') if x else x)
u['lastLogin'] = u['lastLogin'].apply(lambda x:pd.to_datetime(x['$date'],unit='ms') if type(x) != float else x)
# u['createdDate'] = u['createdDate'].apply(lambda x:x['$date'] if x else x)
# u['lastLogin'] = u['lastLogin'].apply(lambda x:x['$date'] if type(x) != float else x)


u.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI


In [10]:
u.shape

(495, 7)

In [11]:
u[u.duplicated()]

,_id,active,createdDate,lastLogin,role,signUpSource,state
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN


In [12]:
u[u['_id'].duplicated()]

,_id,active,createdDate,lastLogin,role,signUpSource,state
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN


In [13]:
u[u['createdDate']>=u['lastLogin']]

,_id,active,createdDate,lastLogin,role,signUpSource,state


In [14]:
u[~u['_id'].duplicated()].groupby('_id').agg(list)

,active,createdDate,lastLogin,role,signUpSource,state
_id,,,,,,
54943462e4b07e684157a532,[True],[2014-12-19 14:21:22.381000],[2021-03-05 16:52:23.204000],[fetch-staff],[nan],[nan]
55308179e4b0eabd8f99caa2,[True],[2015-04-17 03:43:53.186000],[2018-05-07 17:23:40.003000],[consumer],[nan],[WI]
5964eb07e4b03efd0c0f267b,[True],[2017-07-11 15:13:11.771000],[2021-03-04 19:07:49.770000],[fetch-staff],[nan],[IL]
59c124bae4b0299e55b0f330,[True],[2017-09-19 14:07:54.302000],[2021-02-08 16:42:58.117000],[fetch-staff],[nan],[WI]
5a43c08fe4b014fd6b6a0612,[True],[2017-12-27 15:47:27.059000],[2021-02-12 16:22:37.155000],[consumer],[nan],[nan]
...,...,...,...,...,...,...
60255883efa60114d20e5d4e,[True],[2021-02-11 16:17:07.502000],[2021-02-11 16:18:29.219000],[consumer],[Email],[WI]
602558adb5459313e1e9b7ce,[True],[2021-02-11 16:17:49.831000],[2021-02-11 16:17:49.893000],[consumer],[Email],[WI]
602558b1efa60114d20e5dc7,[True],[2021-02-11 16:17:53.738000],[2021-02-11 16:17:53.798000],[consumer],[Email],[WI]


In [15]:
u.query('role == "fetch-staff"')['signUpSource'].value_counts()

Email     42
Google     1
Name: signUpSource, dtype: int64

In [16]:
import sqlite3

con = sqlite3.Connection("db")
r.drop('rewardsReceiptItemList', axis=1).to_sql(name="receipts", con=con, if_exists="append", index=False)
item.to_sql(name="rewardsReceiptItemList", con=con, if_exists="append", index=False)
u.to_sql(name="user", con=con, if_exists="append", index=False)
b.to_sql(name="brand", con=con, if_exists="append", index=False)

In [17]:
pd.read_sql('SELECT * FROM sqlite_master', con)

,type,name,tbl_name,rootpage,sql
0,table,receipts,receipts,2,"CREATE TABLE ""receipts"" (\n""_id"" TEXT,\n ""bon..."
1,table,rewardsReceiptItemList,rewardsReceiptItemList,61,"CREATE TABLE ""rewardsReceiptItemList"" (\n""rece..."
2,table,user,user,344,"CREATE TABLE ""user"" (\n""_id"" TEXT,\n ""active""..."
3,table,brand,brand,358,"CREATE TABLE ""brand"" (\n""_id"" TEXT,\n ""barcod..."


## Answering prompts 

- What are the top 5 brands by receipts scanned for most recent month?
- How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
- When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
- When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
- Which brand has the most spend among users who were created within the past 6 months?
- Which brand has the most transactions among users who were created within the past 6 months?


### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
Since there is no "Accepted" status, it is assumed to be "Finished"

```
SELECT r.rewardsReceiptStatus, ROUND(AVG(r.totalSpent), 2) AS average_spend, SUM(i.quantityPurchased) AS total_purchased
FROM receipts r
    LEFT JOIN rewardsReceiptItemList i
        ON i.receipt_id = r._id
WHERE r.rewardsReceiptStatus in ("FINISHED", "REJECTED")
GROUP BY r.rewardsReceiptStatus
```

In [18]:
#Since there is no "Accepted" status, it is assumed to be "Finished"
pd.read_sql("""
SELECT r.rewardsReceiptStatus, ROUND(AVG(r.totalSpent), 2) AS average_spend, SUM(i.quantityPurchased) AS total_purchased
FROM receipts r
    LEFT JOIN rewardsReceiptItemList i
        ON i.receipt_id = r._id
WHERE r.rewardsReceiptStatus in ("FINISHED", "REJECTED")
GROUP BY r.rewardsReceiptStatus
""", con)

,rewardsReceiptStatus,average_spend,total_purchased
0,FINISHED,1244.58,32704.0
1,REJECTED,19.72,564.0


Using the programming language of your choice (SQL, Python, R, Bash, etc...) identify at least one data quality issue. We are not expecting a full blown review of all the data provided, but instead want to know how you explore and evaluate data of questionable provenance.
Commit your code and findings to the git repository along with the rest of the exercise.

- users table full of duplicates
- rewardsReceiptStatus does not have 'Accepted' (as prompted)
- `barcode` in rewardsReceiptItemList sometimes has 4-digit code or contains non-numeric characters


Construct an email or slack message that is understandable to a product or business leader who isn’t familiar with your day to day work. This part of the exercise should show off how you communicate and reason about data with others. Commit your answers to the git repository along with the rest of your exercise.
- What questions do you have about the data?
- How did you discover the data quality issues?
- What do you need to know to resolve the data quality issues?
- What other information would you need to help you optimize the data assets you're trying to create?
- What performance and scaling concerns do you anticipate in production and how do you plan to address them?

```
Hi there! I'm Kenji from AE team and I was reviewing our database for an automation project and I have couple of quick questions:
1) Going over our `user` data, there are lots of duplicates that is bloating our warehouse - a quick select query will show you that most are duplicated at least once (please see attached). Is there a reason for this? If not, there might be a bug in the pipeline upstream. I can try to find what's causing it in the first place and quickly clean that up.  
2) The documentation also showed that this table should have the `role` column to be 'CONSUMER' exclusively, yet they are 'customer' (plus, 'fetch-staff' can be found here). Is this intended?
If left uncorrected, we might run into some issues in the future with bloated data warehouse and incorrect reports. 
Thanks in advance!
```